<a href="https://colab.research.google.com/github/nagyszeru/DTSA-5800/blob/main/Submisison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
!pip install 'networkx<2.7'
!pip install 'scipy>=1.8.1'
!pip install pyvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.9/748.9 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import gzip
import glob
import os
import shutil
import json
import csv
import networkx as nx
import matplotlib.pyplot as plt
import pyvis
from pyvis.network import Network
from time import sleep
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
import re
import shutil
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("vader_lexicon")
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.sentiment import SentimentIntensityAnalyzer
import string
import itertools
punctuation = string.punctuation
stopwordsset = set(stopwords.words("english"))
stopwordsset.add('rt')
stopwordsset.add("'s")
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Mentions

In [7]:
data_file = gzip.open('drive/MyDrive/nikelululemonadidas_tweets.jsonl.gz')

In [8]:
# Identify unique users in the mention network
uniqueusers = {}

for i, line in enumerate(data_file):
    if i % 10000 == 0:
      print("%s tweets iterated" % i)
    tweet = json.loads(line)
    userwhotweeted = tweet['user']['screen_name']
    idwhotweeted = tweet['user']['id']
    followercount = tweet['user']['followers_count']
    if idwhotweeted in uniqueusers:
      uniqueusers[idwhotweeted]['tweet_count'] += 1
    if idwhotweeted not in uniqueusers:
      uniqueusers[idwhotweeted] = {}
      uniqueusers[idwhotweeted]['tweet_count'] = 1
      uniqueusers[idwhotweeted]['id'] = idwhotweeted
      uniqueusers[idwhotweeted]['followers_count'] = followercount
      uniqueusers[idwhotweeted]['scren_name'] = userwhotweeted

print("There are " + str(len(uniqueusers)) + " unique users.")

0 tweets iterated
10000 tweets iterated
20000 tweets iterated
30000 tweets iterated
40000 tweets iterated
50000 tweets iterated
60000 tweets iterated
70000 tweets iterated
80000 tweets iterated
90000 tweets iterated
100000 tweets iterated
110000 tweets iterated
120000 tweets iterated
130000 tweets iterated
140000 tweets iterated
150000 tweets iterated
160000 tweets iterated
170000 tweets iterated
There are 104772 unique users.


In [ ]:
data_file = gzip.open('drive/MyDrive/nikelululemonadidas_tweets.jsonl.gz')

In [ ]:
userstoinclude = set()

usercount = 0
tweetcount = 0
followercount = 0
tweetcounters = {}
for auser in uniqueusers:
  userwhotweeted = uniqueusers[auser]['scren_name']
  tweetcounters[userwhotweeted] = uniqueusers[auser]['tweet_count']
  if uniqueusers[auser]['tweet_count'] > 2: # if any user has more than 2 tweets 
    if uniqueusers[auser]['followers_count'] > 100000: # if any user has more than 100,000 followers
      usercount += 1
      tweetcount += uniqueusers[auser]['tweet_count']
      followercount += uniqueusers[auser]['followers_count']
      userstoinclude.add(auser)

      
print("There are " + str(len(userstoinclude)) + " 'frequent users' (those with more than 2 tweets and > 100,000 followers), which reduces the num of users to " + str(round(1 - len(userstoinclude) / len(uniqueusers))) + "% of the total users.")
print("These influences cumulatively have  " + str(followercount) + " followers in total.")
print("These influences produced " + str(tweetcount) + " tweets in total.")

There are 100 'frequent users' (those with more than 2 tweets and > 100,000 followers), which reduces the num of users to 1% of the total users.
These influences cumulatively have  92401296 followers in total.
These influences produced 714 tweets in total.


In [ ]:
g=Network(notebook=True, cdn_resources="remote", directed = True, font_color='white', height='1600px', width='100%')

In [ ]:
data_file = gzip.open('drive/MyDrive/nikelululemonadidas_tweets.jsonl.gz')

In [ ]:
# Identify unique users in the mention network
all_tweets = {}
influencers = {}
for i, line in enumerate(data_file):
    # if i > 2:
    #   break
    if i % 10000 == 0:
      print("%s tweets iterated" % i)
    tweet = json.loads(line)
    userwhotweeted = tweet['user']['screen_name']
    idwhotweeted = tweet['user']['id']
    followercount = tweet['user']['followers_count']
    influencers[userwhotweeted] = followercount
    if idwhotweeted in userstoinclude:
      g.add_node(userwhotweeted, color = "blue", shape = "circle")
      users = tweet['entities']['user_mentions']
      if len(users) > 0:
          #break
          for auser in users:
              mention = auser['screen_name']
              if mention != idwhotweeted:
                g.add_node(mention, color = "red", shape = "circle")
                g.add_edge(userwhotweeted, mention, color = "black")  

0 tweets iterated
10000 tweets iterated
20000 tweets iterated
30000 tweets iterated
40000 tweets iterated
50000 tweets iterated
60000 tweets iterated
70000 tweets iterated
80000 tweets iterated
90000 tweets iterated
100000 tweets iterated
110000 tweets iterated
120000 tweets iterated
130000 tweets iterated
140000 tweets iterated
150000 tweets iterated
160000 tweets iterated
170000 tweets iterated


In [ ]:
g.barnes_hut()
g.show('sample.html')
from IPython.core.display import display, HTML
display(HTML('sample.html'))

#Semantic Graph

In [ ]:
# Initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [ ]:
data_file = gzip.open('drive/MyDrive/nikelululemonadidas_tweets.jsonl.gz')

In [ ]:
tweets = []
# Load tweets from gzip-compressed JSON file
for i, line in enumerate(data_file):
  # if i > 10:
  #   break
  if i % 10000 == 0:
    print("%s  tweets iterated" %i)
  tweet = json.loads(line)
  tweets.append(tweet['full_text'])  

0  tweets iterated
10000  tweets iterated
20000  tweets iterated
30000  tweets iterated
40000  tweets iterated
50000  tweets iterated
60000  tweets iterated
70000  tweets iterated
80000  tweets iterated
90000  tweets iterated
100000  tweets iterated
110000  tweets iterated
120000  tweets iterated
130000  tweets iterated
140000  tweets iterated
150000  tweets iterated
160000  tweets iterated
170000  tweets iterated


In [ ]:
custom_stopwords = ['RT', 'rt', 'http', 'https']

In [ ]:
# Filter tweets with positive sentiment
positive_tweets = [tweet for tweet in tweets if sia.polarity_scores(tweet)["compound"] > 0]

# Tokenize positive tweets into words
positive_words = [word_tokenize(tweet) for tweet in positive_tweets]

# Flatten list of lists of words into a single list of words
positive_words = [word for sublist in positive_words for word in sublist]

# Remove stop words and punctuation
stop_words = nltk.corpus.stopwords.words("english")
positive_words = [word for word in positive_words if word.lower() not in stop_words and word.isalpha()]
positive_words = [word for word in positive_words if word.lower() not in custom_stopwords and word.isalpha()]

# Compute word frequency distribution
fdist = nltk.FreqDist(positive_words)

In [ ]:
# Create network with top 20 words in word frequency distribution
net = Network(notebook=True, cdn_resources = "remote", directed = True, height='1600px', width='100%')
for word, freq in fdist.most_common(20):
  if word.lower() in ['adidas', 'nike','lulemon']:
    net.add_node(word, freq=freq, color="red")
  else:
    net.add_node(word, freq=freq, color="blue")


# Add edges between words based on word co-occurrence
for i, (word1, freq1) in enumerate(fdist.most_common(20)):
    for word2, freq2 in fdist.most_common(20)[i + 1:]:
        co_occurrences = sum(word1 in tweet and word2 in tweet for tweet in positive_tweets)
        if co_occurrences > 0:
          if word1.lower() in ['adidas', 'nike','lulemon']:
            net.add_edge(word1, word2, weight=co_occurrences, color="red")
          else:
            net.add_edge(word1, word2, weight=co_occurrences, color="blue")

In [ ]:
net.barnes_hut()
net.show('positive.html')
from IPython.core.display import display, HTML
display(HTML('positive.html'))

In [ ]:
for i, f in enumerate(fdist):
  if i > 20:
    break
  print(f)

Nike
adidas
Xbox
amp
win
chance
Follow
exclusive
XboxSweepstakes
including
kit
Three
icons
like
KingJames
nike
lululemon
nikestore
new
EnesKanter
get


In [ ]:
# Filter tweets with negative sentiment
negative_tweets = [tweet for tweet in tweets if sia.polarity_scores(tweet)["compound"] < 0]

# Tokenize positive tweets into words
negative_words = [word_tokenize(tweet) for tweet in negative_tweets]

# Flatten list of lists of words into a single list of words
negative_words = [word for sublist in negative_words for word in sublist]

# Remove stop words and punctuation
stop_words = nltk.corpus.stopwords.words("english")
negative_words = [word for word in negative_words if word.lower() not in stop_words and word.isalpha()]
negative_words = [word for word in negative_words if word.lower() not in custom_stopwords and word.isalpha()]

# Compute word frequency distribution
fdist = nltk.FreqDist(negative_words)

In [ ]:
# Create network with top 20 words in word frequency distribution
net = Network(notebook=True, cdn_resources = "remote", directed = True, height='1600px', width='100%')
for word, freq in fdist.most_common(20):
  if word.lower() in ['adidas', 'nike','lulemon']:
    net.add_node(word, freq=freq, color="red")
  else:
    net.add_node(word, freq=freq, color="blue")


# Add edges between words based on word co-occurrence
for i, (word1, freq1) in enumerate(fdist.most_common(20)):
    for word2, freq2 in fdist.most_common(20)[i + 1:]:
        co_occurrences = sum(word1 in tweet and word2 in tweet for tweet in positive_tweets)
        if co_occurrences > 0:
          if word1.lower() in ['adidas', 'nike','lulemon']:
            net.add_edge(word1, word2, weight=co_occurrences, color="red")
          else:
            net.add_edge(word1, word2, weight=co_occurrences, color="black")

In [ ]:
net.barnes_hut()
net.show('negative.html')
from IPython.core.display import display, HTML
display(HTML('negative.html'))

#Influencers

In [10]:
data_file = gzip.open('drive/MyDrive/nikelululemonadidas_tweets.jsonl.gz')

In [11]:
userstoinclude = set()
reference = []

usercount = 0
tweetcount = 0
followercount = 0

for auser in uniqueusers:
  userwhotweeted = uniqueusers[auser]['scren_name']
  followers = uniqueusers[auser]['followers_count']
  tweets = uniqueusers[auser]['tweet_count']
  impact = followers * tweets
  #if impact > 18000000:  
  #  if impact < 105000000:
  if followers > 1000:
    if tweets > 125:
      usercount += 1
      tweetcount += uniqueusers[auser]['tweet_count']
      followercount += uniqueusers[auser]['followers_count']
      userstoinclude.add(auser)
      reference.append([userwhotweeted, followers, tweets, impact])
      
print("There are " + str(len(userstoinclude)) + " 'influencers' (those with more than 125 tweets and > 1,000 followers), which reduces the num of users to " + str(round(1 - len(userstoinclude) / len(uniqueusers))) + "% of the total users.")
print("These influences cumulatively have  " + str(followercount) + " followers in total.")
print("These influences produced " + str(tweetcount) + " tweets in total.")

There are 10 'influencers' (those with more than 125 tweets and > 1,000 followers), which reduces the num of users to 1% of the total users.
These influences cumulatively have  40325 followers in total.
These influences produced 9851 tweets in total.


In [12]:
import pandas as pd
df = pd.DataFrame(reference)
df.columns = ['Screen Name', 'Followers', 'Tweets', 'Impact']
df.sort_values('Impact', ascending = False)
final_df = df.sort_values(by=['Impact'], ascending=False)
final_df.reset_index(drop=True, inplace=True)
final_df

,Screen Name,Followers,Tweets,Impact
0,SneakerScouts,18262,7118,129988916
1,Kaya_Alexander5,1744,738,1287072
2,turtlepace5,2794,402,1123188
3,JamarroBrown,7864,133,1045912
4,ShockandAweEnt,1330,398,529340
5,efiorentino31,2303,208,479024
6,vadriano2000,1060,345,365700
7,joshuajhan,1833,172,315276
8,trudyspeaks,2019,126,254394
9,Moonman989,1116,211,235476
